In [11]:
## Project Note
print("This notebook contains the final optimized M5 forecasting pipeline used for the Retail Sales Analytics Dashboard")


This notebook contains the final optimized M5 forecasting pipeline used for the Retail Sales Analytics Dashboard


In [12]:
import os
import gc
import random
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb

warnings.filterwarnings('ignore')

class ProjectConfig:
    DATA_PATH = 'D:/M5 Data'   # <-- IMPORTANT: Set your dataset folder path here
    TRAIN_END = 1913
    FORECAST_HORIZON = 28
    RANDOM_STATE = 42
    
    LGB_PARAMS = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric': 'rmse',
        'subsample': 0.5,
        'subsample_freq': 1,
        'learning_rate': 0.03,
        'num_leaves': 2047,
        'min_data_in_leaf': 4095,
        'feature_fraction': 0.5,
        'max_bin': 100,
        'n_estimators': 1400,
        'boost_from_average': False,
        'verbose': -1,
        'n_jobs': -1
    }

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(ProjectConfig.RANDOM_STATE)


In [13]:
# Load datasets from your D drive

sales = pd.read_csv(f"{ProjectConfig.DATA_PATH}/sales_train_validation.csv")
calendar = pd.read_csv(f"{ProjectConfig.DATA_PATH}/calendar.csv")
prices = pd.read_csv(f"{ProjectConfig.DATA_PATH}/sell_prices.csv")

print("Sales shape:", sales.shape)
print("Calendar shape:", calendar.shape)
print("Prices shape:", prices.shape)


Sales shape: (30490, 1919)
Calendar shape: (1969, 14)
Prices shape: (6841121, 4)


In [14]:
def downcast_dtypes(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage dropped to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    
    return df

In [15]:
def read_data(path):
    print(f"Reading files from {path}...")
    
    calendar = pd.read_csv(f'{path}/calendar.csv')
    calendar = downcast_dtypes(calendar)
    
    prices = pd.read_csv(f'{path}/sell_prices.csv')
    prices = downcast_dtypes(prices)
    
    sales = pd.read_csv(f'{path}/sales_train_validation.csv')
    sales = downcast_dtypes(sales)
    
    return sales, calendar, prices

In [16]:
df_sales, df_calendar, df_prices = read_data(ProjectConfig.DATA_PATH)

Reading files from D:/M5 Data...
Memory usage dropped to  0.12 Mb (41.9% reduction)
Memory usage dropped to 130.48 Mb (37.5% reduction)
Memory usage dropped to 95.00 Mb (78.7% reduction)


In [17]:
def transform_and_merge(sales, calendar, prices, config):
    for day in range(config.FORECAST_HORIZON):
        sales[f'd_{config.TRAIN_END + day + 1}'] = np.nan

    start_idx = max(1, config.TRAIN_END - 1000) 
    value_cols = [c for c in sales.columns if c.startswith('d_') and int(c.split('_')[1]) >= start_idx]
    
    id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    
    data = pd.melt(sales, id_vars=id_cols, value_vars=value_cols, var_name='d', value_name='sales')
    
    calendar = calendar.drop(['weekday', 'wday', 'month', 'year'], axis=1)
    data = data.merge(calendar, on='d', how='left')
    
    data = data.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    
    del calendar, prices
    gc.collect()
    
    return data

In [18]:
def feature_engineering_basic(df):
    df['d_num'] = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
    
    df['date'] = pd.to_datetime(df['date'])
    df['day_of_week'] = df['date'].dt.dayofweek.astype(np.int8)
    df['month'] = df['date'].dt.month.astype(np.int8)
    
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(np.int8)
    
    df['price_momentum'] = df['sell_price'] / df.groupby('id')['sell_price'].transform('mean')
    
    df = df.drop(['date', 'd'], axis=1)
    
    return df

In [19]:
def feature_engineering_lags(df):
    lags = [28, 35, 42, 49, 56]
    for lag in lags:
        df[f'lag_{lag}'] = df.groupby('id')['sales'].shift(lag)
        
    windows = [7, 14, 28, 60]
    for win in windows:
        df[f'rolling_mean_{win}'] = df.groupby('id')['lag_28'].transform(
            lambda x: x.rolling(win).mean())
            
    return df

In [20]:
from sklearn.preprocessing import LabelEncoder

def encode_categoricals(df):
    # Added event_name_2 and event_type_2 to the list
    cat_cols = [
        'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
        'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'
    ]
    
    # Fill NaNs for ALL event columns
    df['event_name_1'] = df['event_name_1'].fillna('NoEvent')
    df['event_type_1'] = df['event_type_1'].fillna('NoEvent')
    df['event_name_2'] = df['event_name_2'].fillna('NoEvent')
    df['event_type_2'] = df['event_type_2'].fillna('NoEvent')
    
    encoder = LabelEncoder()
    for col in cat_cols:
        # We convert to string first to handle any mixed types safely
        df[col] = encoder.fit_transform(df[col].astype(str))
        
    return df

In [21]:
def transform_and_merge(sales, calendar, prices, config):
    # --------------------------------------------------
    # 1. Add future forecast columns
    # --------------------------------------------------
    for day in range(config.FORECAST_HORIZON):
        sales[f'd_{config.TRAIN_END + day + 1}'] = np.nan

    # --------------------------------------------------
    # 2. Limit history to last ~1000 days (MEMORY FIX)
    # --------------------------------------------------
    start_idx = max(1, config.TRAIN_END - 1000)

    value_cols = [
        c for c in sales.columns
        if c.startswith('d_') and int(c.split('_')[1]) >= start_idx
    ]

    id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

    # --------------------------------------------------
    # 3. Melt after limiting columns
    # --------------------------------------------------
    data = pd.melt(
        sales,
        id_vars=id_cols,
        value_vars=value_cols,
        var_name='d',
        value_name='sales'
    )

    # --------------------------------------------------
    # 4. Reduce calendar before merge
    # --------------------------------------------------
    calendar = calendar[
        ['d', 'date', 'wm_yr_wk',
         'event_name_1', 'event_type_1',
         'event_name_2', 'event_type_2']
    ]

    data = data.merge(calendar, on='d', how='left')

    # --------------------------------------------------
    # 5. Reduce prices before merge (CRITICAL)
    # --------------------------------------------------
    prices = prices[['store_id', 'item_id', 'wm_yr_wk', 'sell_price']]

    data = data.merge(
        prices,
        on=['store_id', 'item_id', 'wm_yr_wk'],
        how='left'
    )

    # --------------------------------------------------
    # 6. Cleanup
    # --------------------------------------------------
    del calendar, prices
    gc.collect()

    return data


In [22]:
def perform_split(df, config):
    # --------------------------------------------------
    # 1. Create masks (lightweight)
    # --------------------------------------------------
    train_mask = df['d_num'] <= (config.TRAIN_END - config.FORECAST_HORIZON)
    valid_mask = (
        (df['d_num'] > (config.TRAIN_END - config.FORECAST_HORIZON)) &
        (df['d_num'] <= config.TRAIN_END)
    )

    # --------------------------------------------------
    # 2. Define feature columns FIRST (CRITICAL)
    # --------------------------------------------------
    drop_cols = ['id', 'sales', 'wm_yr_wk', 'd_num']
    feat_cols = [c for c in df.columns if c not in drop_cols]

    # --------------------------------------------------
    # 3. Select only needed columns BEFORE slicing
    # --------------------------------------------------
    df_feat = df[feat_cols + ['sales']]

    # --------------------------------------------------
    # 4. Split (much smaller memory footprint)
    # --------------------------------------------------
    X_tr = df_feat.loc[train_mask, feat_cols]
    y_tr = df_feat.loc[train_mask, 'sales']

    X_val = df_feat.loc[valid_mask, feat_cols]
    y_val = df_feat.loc[valid_mask, 'sales']

    return X_tr, y_tr, X_val, y_val, feat_cols


In [23]:


master_df = pd.read_pickle("processed_dataset.pkl")
print("master_df loaded:", master_df.shape)


master_df loaded: (31374210, 30)


In [24]:
X_tr, y_tr, X_val, y_val, feats = perform_split(master_df, ProjectConfig)


# =========================================================
MODEL TRAINING
# =========================================================

In [25]:
def run_lgbm(X_tr, y_tr, X_val, y_val, config):
    dtrain = lgb.Dataset(X_tr, y_tr)
    dvalid = lgb.Dataset(X_val, y_val)
    
    model = lgb.train(
        config.LGB_PARAMS,
        dtrain,
        valid_sets=[dtrain, dvalid],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(5),
            lgb.log_evaluation(100)
        ]
    )
    
    return model

In [ ]:
X_train, y_train, X_valid, y_valid, features = perform_split(master_df, ProjectConfig)

model = run_lgbm(X_train, y_train, X_valid, y_valid, ProjectConfig)

model.save_model('lgbm_model_v1.txt')

Training until validation scores don't improve for 5 rounds


In [ ]:
def predict_forecast(df, model, config, feats):
    pred_mask = df['d_num'] > config.TRAIN_END
    X_test = df[pred_mask][feats]
    
    preds = model.predict(X_test)
    
    output = df[pred_mask][['id', 'd_num']].copy()
    output['sales'] = preds
    
    return output

In [ ]:
def format_submission(output_df, config):
    output_df = output_df.pivot(index='id', columns='d_num', values='sales').reset_index()
    
    output_df.columns = ['id'] + [f'F{i}' for i in range(1, config.FORECAST_HORIZON + 1)]
    
    eval_output = output_df.copy()
    eval_output['id'] = eval_output['id'].str.replace('_validation', '_evaluation')
    
    final_sub = pd.concat([output_df, eval_output], axis=0, sort=False)
    
    return final_sub

In [ ]:
forecast_df = predict_forecast(master_df, model, ProjectConfig, features)

submission = format_submission(forecast_df, ProjectConfig)

submission.to_csv('submission.csv', index=False)

print("Submission file generated successfully.")
print(submission.head())

# =========================================================
# ML Model Evaluation on Validation Data
# =========================================================

In [ ]:
# =========================================================
# LightGBM – 28 Day Evaluation
# =========================================================

# Predict on validation set
y_ml_pred = model.predict(X_valid)

ml_mae_28 = mean_absolute_error(y_valid, y_ml_pred)
ml_rmse_28 = np.sqrt(mean_squared_error(y_valid, y_ml_pred))

print("LightGBM MAE (28 days):", ml_mae_28)
print("LightGBM RMSE (28 days):", ml_rmse_28)


# =========================================================
# Baseline Model Comparison 
# =========================================================

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# =========================================================
# Full 28-Day Validation Window
# =========================================================

valid_mask = (
    (master_df["d_num"] > (ProjectConfig.TRAIN_END - ProjectConfig.FORECAST_HORIZON)) &
    (master_df["d_num"] <= ProjectConfig.TRAIN_END)
)

val_df = master_df[valid_mask][["id", "d_num", "sales"]].copy()

print(val_df.head())
print("Validation days:", val_df["d_num"].nunique())


In [ ]:
# =========================================================
# Baseline Model Comparison – Naive Baseline
# =========================================================

last_sales = (
    master_df[master_df["d_num"] <= (ProjectConfig.TRAIN_END - ProjectConfig.FORECAST_HORIZON)]
    .groupby("id")["sales"]
    .last()
)

# Create 28-day naive forecast
naive_forecast = val_df.copy()
naive_forecast["pred"] = naive_forecast["id"].map(last_sales)

# Metrics
naive_mae_28 = mean_absolute_error(naive_forecast["sales"], naive_forecast["pred"])
naive_rmse_28 = np.sqrt(mean_squared_error(naive_forecast["sales"], naive_forecast["pred"]))

print("Naive MAE (28 days):", naive_mae_28)
print("Naive RMSE (28 days):", naive_rmse_28)

In [ ]:
# =========================================================
# Moving Average (28 days) – 28 Day Evaluation
# =========================================================

# Average of last 28 training days per item
train_data = master_df[master_df["d_num"] <= (ProjectConfig.TRAIN_END - ProjectConfig.FORECAST_HORIZON)]

ma_28 = (
    train_data
    .sort_values("d_num")
    .groupby("id")
    .tail(28)
    .groupby("id")["sales"]
    .mean()
)

# Create forecast
ma_forecast = val_df.copy()
ma_forecast["pred"] = ma_forecast["id"].map(ma_28)

# Metrics
ma_mae_28 = mean_absolute_error(ma_forecast["sales"], ma_forecast["pred"])
ma_rmse_28 = np.sqrt(mean_squared_error(ma_forecast["sales"], ma_forecast["pred"]))

print("Moving Avg MAE (28 days):", ma_mae_28)
print("Moving Avg RMSE (28 days):", ma_rmse_28)


# =========================================================
# Final Model Comparison (Baseline vs ML)
# =========================================================

In [ ]:
# =========================================================
# Final 28-Day Model Comparison
# =========================================================

comparison_28_df = pd.DataFrame({
    "Model": [
        "Naive Baseline",
        "Moving Average (28 days)",
        "LightGBM (Tweedie)"
    ],
    "MAE (28 days)": [
        naive_mae_28,
        ma_mae_28,
        ml_mae_28
    ],
    "RMSE (28 days)": [
        naive_rmse_28,
        ma_rmse_28,
        ml_rmse_28
    ]
})

comparison_28_df


In [ ]:
rmse_improvement = (ma_rmse_28 - ml_rmse_28) / ma_rmse_28 * 100
mae_improvement = (ma_mae_28 - ml_mae_28) / ma_mae_28 * 100

print(f"RMSE Improvement over MA: {rmse_improvement:.2f}%")
print(f"MAE Improvement over MA: {mae_improvement:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

def plot_item_forecast(master_df, forecast_df, item_id, config):
    history_mask = (master_df['id'] == item_id) & (master_df['d_num'] <= config.TRAIN_END)
    # Show last 90 days of history for clarity
    history_mask = history_mask & (master_df['d_num'] > config.TRAIN_END - 90)
    history = master_df[history_mask]
    
    forecast = forecast_df[forecast_df['id'] == item_id]
    
    plt.figure(figsize=(14, 7))
    
    plt.plot(history['d_num'], history['sales'], label='Historical Sales (Last 90 Days)', color='#1f77b4', linewidth=2)
    
    plt.plot(forecast['d_num'], forecast['sales'], label='Forecast (Next 28 Days)', color='#ff7f0e', linestyle='--', linewidth=2.5)
    
    plt.axvline(x=config.TRAIN_END, color='grey', linestyle=':', linewidth=1.5, label='Forecast Start')
    
    plt.title(f'Sales Forecast for Item: {item_id}', fontsize=16)
    plt.xlabel('Day Number', fontsize=12)
    plt.ylabel('Sales Volume', fontsize=12)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

def visualize_results(master_df, forecast_df, config, num_items=3):
    unique_ids = forecast_df['id'].unique()
    random_items = np.random.choice(unique_ids, num_items, replace=False)
    
    for item_id in random_items:
        plot_item_forecast(master_df, forecast_df, item_id, config)

In [ ]:
# Ensure you have run all previous sections first
visualize_results(master_df, forecast_df, ProjectConfig, num_items=3)

In [ ]:
item_meta = sales[['id','item_id','dept_id','cat_id','store_id','state_id']]
item_meta = item_meta.drop_duplicates()
item_meta.to_csv("item_master.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm


In [ ]:
DATA_PATH = "D:/M5 Data"

sales = pd.read_csv(
    f"{DATA_PATH}/sales_train_evaluation.csv"
)

calendar = pd.read_csv(
    f"{DATA_PATH}/calendar.csv",
    usecols=["d", "date"]
)

print("Sales shape:", sales.shape)
print("Calendar shape:", calendar.shape)


In [ ]:
id_cols = ["id"]

sales_long = sales.melt(
    id_vars=id_cols,
    var_name="d",
    value_name="sales"
)

print(sales_long.head())
print("Long format shape:", sales_long.shape)


In [ ]:
sales_long = sales_long.merge(calendar, on="d", how="left")

sales_long["date"] = pd.to_datetime(sales_long["date"])

print(sales_long.head())


In [ ]:
last_date = sales_long["date"].max()
cutoff_date = last_date - pd.Timedelta(days=90)

historical_90_days = sales_long[
    sales_long["date"] >= cutoff_date
][["id", "date", "sales"]]

historical_90_days.to_csv(
    "historical_90_days.csv",
    index=False
)

print("historical_90_days.csv saved")


In [ ]:
sales_long["year"] = sales_long["date"].dt.year
sales_long["month"] = sales_long["date"].dt.month

historical_monthly = (
    sales_long
    .groupby(["id", "year", "month"], as_index=False)
    ["sales"]
    .sum()
)

historical_monthly.to_csv(
    "historical_monthly.csv",
    index=False
)

print("historical_monthly.csv saved")
